In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy
import os 
import pathlib
import argparse
from PIL import Image

import easydict

In [2]:
def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs

    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [3]:
class IgnoreLabelDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index]

    def __len__(self):
        return len(self.orig)

In [4]:
def read_data(pred_path):
    pred = []
    path = pathlib.Path(pred_path)
    files = list(path.glob('*.jpg')) + list(path.glob('*.png'))
    # num = len(files)

    for name in files:
        pred.append(np.asarray(Image.open(os.path.join(pred_path, name))))

    pred = np.asarray(pred)
    # if images are gray
    if len(pred.shape) == 3:
        pred = pred[:,:,:,np.newaxis]
        pred = np.concatenate([pred,pred,pred], axis=3)
    pred = pred.transpose((0, 3, 1, 2))
    pred = pred.astype(np.float32) / 255.0

    return pred

In [5]:
args = easydict.EasyDict({
    "path": "D:/Jupyter/museong/1. Word Hierarchy-based Text Augmentation for Text to Image Synthesis/2. Synonym Replacement(5)/models/netG_epoch_130/valid/single",
})

In [6]:
if __name__ == '__main__':
    #parser = argparse.ArgumentParser()
    #parser.add_argument('-p', '--path', type=str, default=None)
    args #= parser.parse_args()
    
    print('Read data...')
    data = read_data(args.path)

    ite_data = IgnoreLabelDataset(data)

    print ("Calculating Inception Score...")
    print (inception_score(ite_data, cuda=True, batch_size=10, resize=True, splits=10))

Read data...
Calculating Inception Score...


D:\Anaconda3\envs\torchgan\lib\site-packages\torch\nn\functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
D:\Anaconda3\envs\torchgan\lib\site-packages\ipykernel\__main__.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(25.323009372343382, 0.46281519798365933)
